# Reflection 패턴: 자기 성찰 에이전트

이 노트북에서는 **Reflection 패턴**을 사용하여 **스스로 결과물을 개선**하는 에이전트를 만듭니다.

## Reflection이란?

```
┌────────────────────────────────────────────────────────────────────┐
│                    Reflection (자기 성찰) 패턴                     │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반적인 생성:                                                   │
│   질문 → LLM → 답변 (1회 생성 후 끝)                              │
│                                                                    │
│   Reflection 패턴:                                                 │
│   질문 → LLM(생성) → 결과물                                       │
│            ↑           │                                          │
│            │           ▼                                          │
│            └─── LLM(평가) ← 개선할 점 찾기                        │
│                                                                    │
│   → 생성과 평가를 반복하여 품질 향상!                             │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 아키텍처

```
┌────────────────────────────────────────────────────────────────────┐
│                    Reflection 그래프 구조                          │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   START                                                            │
│     │                                                              │
│     ▼                                                              │
│   ┌──────────┐                                                     │
│   │ generate │  ← 에세이/코드/답변 생성                           │
│   └────┬─────┘                                                     │
│        │                                                           │
│     should_continue?                                               │
│        │╲                                                          │
│        │ ╲ (계속)                                                  │
│        │  ╲                                                        │
│   (종료) ▼  ▼                                                      │
│   END    ┌──────────┐                                              │
│          │ reflect  │  ← 결과물 평가 및 피드백                    │
│          └────┬─────┘                                              │
│               │                                                    │
│               └──────────────────────────────────────┐             │
│                                                      │             │
│                                                  generate          │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama langgraph

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. LLM 설정

In [ ]:
from langchain_ollama import ChatOllama

model = ChatOllama(model='llama3.2')

print("✅ LLM 설정 완료")

# 3. State 정의

In [ ]:
from typing import Annotated, TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

class State(TypedDict):
    """
    Reflection 패턴의 상태
    
    messages: 대화 기록
      - 사용자 요청
      - 생성된 결과물 (AI)
      - 평가 피드백 (Human으로 변환)
      - 개선된 결과물 (AI)
      - ... 반복
    """
    messages: Annotated[list[BaseMessage], add_messages]

print("✅ State 정의 완료")

# 4. 프롬프트 정의

생성과 평가에 사용할 시스템 프롬프트를 정의합니다.

In [ ]:
from langchain_core.messages import SystemMessage

# 생성용 프롬프트
generate_prompt = SystemMessage(
    content='당신은 훌륭한 3단락 에세이를 작성하는 임무를 가진 에세이 어시스턴트입니다.'
    '사용자의 요청에 맞춰 최상의 에세이를 작성하세요.'
    '사용자가 비평을 제공하면, 이전 시도에 대한 수정 버전을 응답하세요.'
)

# 평가용 프롬프트
reflection_prompt = SystemMessage(
    content='당신은 에세이 제출물을 평가하는 교사입니다. 사용자의 제출물에 대해 비평과 추천을 생성하세요.'
    '길이, 깊이, 스타일 등과 같은 구체적인 요구사항을 포함한 자세한 추천을 제공하세요.'
)

print("✅ 프롬프트 정의 완료")
print("   - generate_prompt: 에세이 작성")
print("   - reflection_prompt: 에세이 평가")

# 5. 노드 정의

```
┌────────────────────────────────────────────────────────────────────┐
│                    메시지 역할 변환 트릭                            │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   reflect 노드에서 메시지 역할을 뒤집는 이유:                      │
│                                                                    │
│   [원래]                    [변환 후]                              │
│   Human: 에세이 써줘        Human: 에세이 써줘                     │
│   AI: 에세이 내용...   →    Human: 에세이 내용...                  │
│                                                                    │
│   → AI가 자신이 쓴 에세이를 '사용자가 제출한 것'처럼               │
│     평가하도록 만듦!                                               │
│                                                                    │
│   그 후 평가 결과를 HumanMessage로 반환                            │
│   → generate 노드가 피드백으로 받아들임                            │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

def generate(state: State) -> State:
    """
    생성 노드: 에세이 작성 (또는 개선)
    
    1. 시스템 프롬프트 + 대화 기록 조합
    2. LLM으로 에세이 생성
    3. 결과를 AIMessage로 반환
    """
    answer = model.invoke([generate_prompt] + state['messages'])
    return {'messages': [answer]}


def reflect(state: State) -> State:
    """
    평가 노드: 에세이 비평
    
    1. 메시지 역할 변환 (AI → Human, Human → AI)
    2. 평가 프롬프트로 비평 생성
    3. 비평을 HumanMessage로 반환 (피드백으로 사용)
    """
    # 역할 변환 매핑
    cls_map = {AIMessage: HumanMessage, HumanMessage: AIMessage}
    
    # 첫 번째 메시지(원래 요청)는 유지, 나머지는 역할 변환
    translated = [reflection_prompt, state['messages'][0]] + [
        cls_map[msg.__class__](content=msg.content) 
        for msg in state['messages'][1:]
    ]
    
    # 평가 생성
    answer = model.invoke(translated)
    
    # 평가 결과를 HumanMessage로 반환 (피드백으로 취급)
    return {'messages': [HumanMessage(content=answer.content)]}


print("✅ 노드 정의 완료")
print("   - generate: 에세이 생성/개선")
print("   - reflect: 에세이 평가")

# 6. 조건 함수 정의

In [ ]:
from langgraph.graph import END

def should_continue(state: State):
    """
    반복 여부 결정
    
    메시지 수로 반복 횟수 제어:
    - 초기: 1개 (사용자 요청)
    - 1회차 후: 3개 (요청 + 에세이 + 피드백)
    - 2회차 후: 5개 (+ 개선 에세이 + 피드백)
    - 3회차 후: 7개 → 6개 초과 → 종료
    """
    if len(state['messages']) > 6:
        return END  # 3회 반복 후 종료
    else:
        return 'reflect'  # 계속 평가

print("✅ should_continue 함수 정의 완료")
print("   3회 반복 후 종료")

# 7. 그래프 구성

In [ ]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(State)

# 노드 추가
builder.add_node('generate', generate)
builder.add_node('reflect', reflect)

# 엣지 추가
builder.add_edge(START, 'generate')              # 시작 → 생성
builder.add_conditional_edges('generate', should_continue)  # 조건부 분기
builder.add_edge('reflect', 'generate')          # 평가 → 생성 (루프)

graph = builder.compile()

print("✅ 그래프 컴파일 완료")

In [ ]:
# 그래프 구조 시각화
print("=== Reflection 그래프 구조 ===")
print(graph.get_graph().draw_mermaid())

# 8. Reflection 에이전트 실행

In [ ]:
# 에세이 주제 설정
initial_state = {
    'messages': [
        HumanMessage(
            content="오늘날 '어린 왕자'가 왜 중요한지에 대해 에세이를 작성하세요."
        )
    ]
}

print("=== Reflection 에이전트 실행 ===")
print(f"주제: {initial_state['messages'][0].content}\n")

iteration = 0
for output in graph.stream(initial_state):
    if 'generate' in output:
        iteration += 1
        print(f"\n{'='*60}")
        print(f"📝 생성 #{iteration}")
        print("="*60)
        content = output['generate']['messages'][-1].content
        print(f"{content[:500]}..." if len(content) > 500 else content)
        
    elif 'reflect' in output:
        print(f"\n{'─'*60}")
        print(f"🔍 평가 #{iteration}")
        print("─"*60)
        content = output['reflect']['messages'][-1].content
        print(f"{content[:500]}..." if len(content) > 500 else content)

In [ ]:
# 최종 결과 확인
result = graph.invoke(initial_state)

print("=== 최종 에세이 ===")
# 마지막 AI 메시지 찾기
for msg in reversed(result['messages']):
    if isinstance(msg, AIMessage):
        print(msg.content)
        break

---

## 정리: Reflection 패턴

### 핵심 개념

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Reflection 패턴 동작 원리                        │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   1. generate: 결과물 생성                                          │
│   2. reflect: 결과물 평가 (역할 변환 트릭 사용)                     │
│   3. generate: 피드백 반영하여 개선                                 │
│   4. 반복...                                                        │
│                                                                     │
│   핵심 트릭:                                                        │
│   • AI 응답을 Human 메시지로 변환 → '제출물'처럼 평가              │
│   • 평가 결과를 Human 메시지로 반환 → '피드백'으로 전달            │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 핵심 코드

```python
# 역할 변환
cls_map = {AIMessage: HumanMessage, HumanMessage: AIMessage}
translated = [prompt, original_request] + [
    cls_map[msg.__class__](content=msg.content) 
    for msg in messages[1:]
]

# 피드백으로 반환
return {'messages': [HumanMessage(content=answer.content)]}
```

### 활용 사례

| 사례 | 설명 |
|------|------|
| **글쓰기** | 에세이, 보고서 품질 향상 |
| **코드** | 코드 리뷰 및 개선 |
| **번역** | 번역 품질 검토 및 수정 |
| **요약** | 요약 정확도 향상 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model = ChatOpenAI(model='gpt-4o-mini')

# 변경
model = ChatOllama(model='llama3.2')
```

## 다음 단계

**Subgraph**를 사용하여 그래프 내에 그래프를 넣는 방법을 배웁니다. (02번 노트북)